In [1]:
import pandas as pd
import array, csv, datetime, glob, math, os, random

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]

def YearMonthDayToEpoch(year, month, day):
    return (datetime.datetime(int(year), int(month), int(day)) - datetime.datetime(1970, 1, 1)).total_seconds()

class GoogleSheetRow():
    columns = [
        'Enabled', 'Share link identifier', 'Category', 'Name', 'Credits', 'Base layer', 'Custom slider ticks', 'Start date',
        'End date', 'Step', 'URL', 'Scaling', 'Color Scaling', 'Map Type', 'Color', 'External GeoJSON', 'Name Key', 'Show Graph',
        'Graph Title', 'Graph Plot Colors', 'Graph Plots First Visible', 'Graph X-Axis Label', 'Graph Y-Axis Label', 'Graph Y-Axis Min',
        'Graph Y-Axis Max', 'Graph X-Axis Label Interval', 'Legend Content', 'Legend Key', 'Load Data Function', 'Set Data Function', 
        'Number of Levels', 'Number of Attributes', 'Vertex Shader', 'Fragment Shader', 'Draw Function', 'Playback Rate',
        'Master Playback Rate', 'Colormap Src', 'Layer Description', 'Featured Themes', 'Draw Options', 'Set Data Options', 'Extras Options',
        'Draw Order', 'Timeline Type', 'Layer Constraints', 'Mapbox', 'Draw Layer Function'
    ]
    key_values = {}
    def __init__(self, _dict = {}):
        for column in self.columns:
            self.key_values[column] = ''
        for key in _dict.keys():
            if key in self.columns:
                self.key_values[key] = _dict[key]
    def set_column(self, key, value):
        if key in self.columns:
            self.key_values[key] = value
    def print_row(self):
        row = []
        for column in self.columns:
            row.append(self.key_values[column])
        print("\t".join(row))
        


In [2]:
fnames = sorted(glob.glob('../data/making-a-fires-map/DL_FIRE_SV-C2_478326/*.csv'))
fnames

['../data/making-a-fires-map/DL_FIRE_SV-C2_478326/brazil_fires.csv',
 '../data/making-a-fires-map/DL_FIRE_SV-C2_478326/fire_archive_SV-C2_478326.csv',
 '../data/making-a-fires-map/DL_FIRE_SV-C2_478326/fire_nrt_SV-C2_478326.csv']

In [3]:
df = pd.read_csv(fnames[0])

In [4]:
df.columns

Index(['latitude', 'longitude', 'acq_date', 'confidence'], dtype='object')

In [5]:
df['confidence'].unique()

array(['n', 'h', 'l'], dtype=object)

In [6]:
points = []

length = len(fnames)

for i in range(length):
    df = pd.read_csv(fnames[i])
    for idx, row in df.iterrows():
        if row['confidence'] != 'l':
            x,y = LonLatToPixelXY((row['longitude'], row['latitude']))
            year,m,d = row['acq_date'].split('-')
            epoch = YearMonthDayToEpoch(year,m,d)
            points += [x,y,epoch]
        

In [7]:
len(points)/3

282035.0

In [8]:
array.array('f', points).tofile(open('../data/making-a-fires-map/making_a_fires_map.bin', 'wb'))

In [9]:
!du -hs ../data/making-a-fires-map/making_a_fires_map.bin

3.2M	../data/making-a-fires-map/making_a_fires_map.bin


In [10]:
!scp ../data/making-a-fires-map/making_a_fires_map.bin timemachine2:/t/earthtime.org/app/data/making-earthtime-maps


ssh: Could not resolve hostname timemachine2: nodename nor servname provided, or not known
scp: Connection closed


In [11]:
# Share link identifiers should be unique. Would be great if a DB just handled this... 
share_link_identifer = 'making_a_fires_map_example_1_' + str(random.getrandbits(16))
category = 'Making EarthTime Maps'
name = 'Making a fires map (Example 1)'
url = 'https://tiles.earthtime.org/making-earthtime-maps/making_a_fires_map.bin'
layer_args = {
    'Enabled': 'TRUE',
    'Share link identifier': share_link_identifer,
    'Category': category, 
    'Name': name, 
    'Credits': 'NASA', 
    'Base layer': 'bdrk', 
    'Start date': '20180101',
    'End date': '20231231', 
    'URL': url, 
    'Map Type': 'vector',
    'Load Data Function': 'WebGLVectorTile2.prototype._loadData',
    'Set Data Function': 'WebGLVectorTile2.prototype._setBufferData',
    'Number of Levels': '0',
    'Number of Attributes': '3',
    'Vertex Shader': 'WebGLVectorTile2.viirsVertexShader',
    'Fragment Shader': 'WebGLVectorTile2.viirsFragmentShader',
    'Draw Function': 'WebGLVectorTile2.prototype._drawViirs',
    'Playback Rate': '8',
    'Master Playback Rate': '16',
    'Draw Options': '{"color": [209, 56, 18]}',
    'Draw Order': '700',
}
row = GoogleSheetRow(layer_args)
row.print_row()

TRUE	making_a_fires_map_example_1_49705	Making EarthTime Maps	Making a fires map (Example 1)	NASA	bdrk		20180101	20231231		https://tiles.earthtime.org/making-earthtime-maps/making_a_fires_map.bin			vector															WebGLVectorTile2.prototype._loadData	WebGLVectorTile2.prototype._setBufferData	0	3	WebGLVectorTile2.viirsVertexShader	WebGLVectorTile2.viirsFragmentShader	WebGLVectorTile2.prototype._drawViirs	8	16				{"color": [209, 56, 18]}			700				


In [12]:
# Customizing the time span
share_link_identifer = 'making_a_fires_map_example_2_' + str(random.getrandbits(16))
category = 'Making EarthTime Maps'
name = 'Making a fires map (Example 2)'
span = 1*24*60*60 # (Number of days) * (24 hours) * (60 minutes) * (60 seconds) 
# This should be wherever you uploaded your data to
url = 'https://tiles.earthtime.org/making-earthtime-maps/making_a_fires_map.bin'
layer_args = {
    'Enabled': 'TRUE',
    'Share link identifier': share_link_identifer,
    'Category': category, 
    'Name': name, 
    'Credits': 'NASA', 
    'Base layer': 'bdrk', 
    'Start date': '20180101',
    'End date': '20230101', 
    'URL': url, 
    'Map Type': 'vector',
    'Load Data Function': 'WebGLVectorTile2.prototype._loadData',
    'Set Data Function': 'WebGLVectorTile2.prototype._setBufferData',
    'Number of Levels': '0',
    'Number of Attributes': '3',
    'Vertex Shader': 'WebGLVectorTile2.viirsVertexShader',
    'Fragment Shader': 'WebGLVectorTile2.viirsFragmentShader',
    'Draw Function': 'WebGLVectorTile2.prototype._drawViirs',
    'Playback Rate': '8',
    'Master Playback Rate': '16',
    'Draw Options': '{"color": [187, 16, 8], "span": '+str(span)+'}',
    'Draw Order': '700',
}
row = GoogleSheetRow(layer_args)
row.print_row()

TRUE	making_a_fires_map_example_2_53954	Making EarthTime Maps	Making a fires map (Example 2)	NASA	bdrk		20180101	20230101		https://tiles.earthtime.org/making-earthtime-maps/making_a_fires_map.bin			vector															WebGLVectorTile2.prototype._loadData	WebGLVectorTile2.prototype._setBufferData	0	3	WebGLVectorTile2.viirsVertexShader	WebGLVectorTile2.viirsFragmentShader	WebGLVectorTile2.prototype._drawViirs	8	16				{"color": [187, 16, 8], "span": 86400}			700				


In [14]:
points_by_year = {
    '2018': [],
    '2019': [],
    '2020': [],
    '2021': [],
    '2022': [],
    '2023': [], 
    '2024': [],
}

for i in range(length):
    df = pd.read_csv(fnames[i])
    for idx, row in df.iterrows():
        if row['confidence'] != 'l':
            x,y = LonLatToPixelXY((row['longitude'], row['latitude']))
            year,m,d = row['acq_date'].split('-')
            epoch = YearMonthDayToEpoch(year,m,d)
            points_by_year[year] += [x,y,epoch]


In [15]:
for key in points_by_year.keys():
    print(key, len(points_by_year[key]))

2018 18030
2019 50412
2020 102519
2021 42558
2022 69321
2023 70317
2024 492948


In [16]:
for key in points_by_year.keys():
    array.array('f', points_by_year[key]).tofile(open('../data/making-a-fires-map/making_a_fires_map_'+ key +'.bin', 'wb'))
    cmd = f'scp ../data/making-a-fires-map/making_a_fires_map_{key}.bin timemachine2:/t/earthtime.org/app/data/making-earthtime-maps'
    !$cmd


ssh: Could not resolve hostname timemachine2: nodename nor servname provided, or not known
scp: Connection closed
ssh: Could not resolve hostname timemachine2: nodename nor servname provided, or not known
scp: Connection closed
ssh: Could not resolve hostname timemachine2: nodename nor servname provided, or not known
scp: Connection closed
ssh: Could not resolve hostname timemachine2: nodename nor servname provided, or not known
scp: Connection closed
ssh: Could not resolve hostname timemachine2: nodename nor servname provided, or not known
scp: Connection closed
ssh: Could not resolve hostname timemachine2: nodename nor servname provided, or not known
scp: Connection closed
ssh: Could not resolve hostname timemachine2: nodename nor servname provided, or not known
scp: Connection closed


In [17]:
# Produce 6 layers showing total for each year
category = 'Making EarthTime Maps'
span = 365*24*60*60 # (Number of days) * (24 hours) * (60 minutes) * (60 seconds) 
layer_args = {
    'Enabled': 'TRUE',
    'Share link identifier': '',
    'Category': category, 
    'Name': '', 
    'Credits': 'NASA', 
    'Base layer': 'bdrk', 
    'Start date': '',
    'End date': '', 
    'URL': '', 
    'Map Type': 'vector',
    'Load Data Function': 'WebGLVectorTile2.prototype._loadData',
    'Set Data Function': 'WebGLVectorTile2.prototype._setBufferData',
    'Number of Levels': '0',
    'Number of Attributes': '3',
    'Vertex Shader': 'WebGLVectorTile2.viirsVertexShader',
    'Fragment Shader': 'WebGLVectorTile2.viirsFragmentShader',
    'Draw Function': 'WebGLVectorTile2.prototype._drawViirs',
    'Playback Rate': '',
    'Master Playback Rate': '',
    'Draw Options': '{"color": [187, 16, 8], "span": '+str(span)+'}',
    'Draw Order': '700',
}
for key in points_by_year.keys():
    share_link_identifer = f'making_a_fires_map_example_{key}_' + str(random.getrandbits(16))
    name = f'Making a fires map ({key})'
    url = f'https://tiles.earthtime.org/making-earthtime-maps/making_a_fires_map_{key}.bin'
    layer_args['Share link identifier'] = share_link_identifer
    layer_args['Name'] = name
    layer_args['URL'] = url
    layer_args['Start date'] = f'{key}0101'
    layer_args['End date'] = f'{key}1231'
    row = GoogleSheetRow(layer_args)
    row.print_row()

TRUE	making_a_fires_map_example_2018_14166	Making EarthTime Maps	Making a fires map (2018)	NASA	bdrk		20180101	20181231		https://tiles.earthtime.org/making-earthtime-maps/making_a_fires_map_2018.bin			vector															WebGLVectorTile2.prototype._loadData	WebGLVectorTile2.prototype._setBufferData	0	3	WebGLVectorTile2.viirsVertexShader	WebGLVectorTile2.viirsFragmentShader	WebGLVectorTile2.prototype._drawViirs						{"color": [187, 16, 8], "span": 31536000}			700				
TRUE	making_a_fires_map_example_2019_12587	Making EarthTime Maps	Making a fires map (2019)	NASA	bdrk		20190101	20191231		https://tiles.earthtime.org/making-earthtime-maps/making_a_fires_map_2019.bin			vector															WebGLVectorTile2.prototype._loadData	WebGLVectorTile2.prototype._setBufferData	0	3	WebGLVectorTile2.viirsVertexShader	WebGLVectorTile2.viirsFragmentShader	WebGLVectorTile2.prototype._drawViirs						{"color": [187, 16, 8], "span": 31536000}			700				
TRUE	making_a_fires_map_example_2020_11367	Making EarthT